In [3]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from basemodels.models import Match
import pandas as pd

data = Match.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df

,id,startTimestamp,slug,tournament_id,season_id,roundInfo_round,status_type,homeTeam_id,awayTeam_id,homeScore_current,homeScore_period1,homeScore_period2,homeScore_normaltime,awayScore_current,awayScore_period1,awayScore_period2,awayScore_normaltime
0,11907436,2024-10-21 12:00:00+00:00,djurgardens-if-hammarby-if,40,57284,27,finished,1758,1759,2,2,0,2,0,0,0,0
1,11907437,2024-10-21 17:10:00+00:00,aik-ifk-goteborg,40,57284,27,finished,1761,1764,1,0,1,1,2,1,1,2
2,11907438,2024-10-21 17:00:00+00:00,if-brommapojkarna-ifk-norrkoping,40,57284,27,finished,1769,1787,1,1,0,1,1,1,0,1
3,11907440,2024-10-20 14:30:00+00:00,ifk-varnamo-gais-goteborg,40,57284,27,finished,1786,1842,0,0,0,0,0,0,0,0
4,11907442,2024-10-20 12:00:00+00:00,kalmar-ff-mjallby-aif,40,57284,27,finished,1891,1783,0,0,0,0,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,12991468,2024-10-20 20:30:00+00:00,sol-de-america-cerro-porteno,11541,63769,16,finished,36439,5991,0,0,0,0,1,1,0,1
478,12991470,2024-10-20 23:00:00+00:00,olimpia-libertad,11541,63769,16,finished,5996,5997,0,0,0,0,1,0,1,1
479,12992167,2024-10-20 17:15:00+00:00,ksc-lokeren-temse-rfc-seraing,9,61412,8,finished,371484,204712,0,0,0,0,1,1,0,1
480,13008795,2024-10-22 23:30:00+00:00,atlanta-united-cf-montreal,242,57317,0,notstarted,22006,243211,0,0,0,0,0,0,0,0


In [10]:
import pandas as pd

# Puan durumu hesaplamaya başla
def puan_durumu(df,  season_id):
    # İlgili sezon ve turnuvadaki maçları filtrele
    df = df[ (df['season_id'] == season_id) & (df['status_type'] == 'finished')]

    # Tüm takımları bul
    teams = pd.concat([df['homeTeam_id'], df['awayTeam_id']]).unique()

    # Puan durumu için gerekli sütunları oluştur
    puan_durumu = pd.DataFrame(teams, columns=['team_id'])
    puan_durumu['W'] = 0  # Galibiyet
    puan_durumu['D'] = 0  # Beraberlik
    puan_durumu['L'] = 0  # Mağlubiyet
    puan_durumu['GF'] = 0  # Atılan Goller
    puan_durumu['GA'] = 0  # Yenen Goller
    puan_durumu['Pts'] = 0  # Puan

    # Her maç için takımların performansını güncelle
    for _, row in df.iterrows():
        home_team = row['homeTeam_id']
        away_team = row['awayTeam_id']
        home_score = row['homeScore_current']
        away_score = row['awayScore_current']

        # Atılan ve yenilen golleri güncelle
        puan_durumu.loc[puan_durumu['team_id'] == home_team, 'GF'] += home_score
        puan_durumu.loc[puan_durumu['team_id'] == home_team, 'GA'] += away_score
        puan_durumu.loc[puan_durumu['team_id'] == away_team, 'GF'] += away_score
        puan_durumu.loc[puan_durumu['team_id'] == away_team, 'GA'] += home_score

        # Maç sonucuna göre puan, galibiyet, beraberlik, mağlubiyet güncelle
        if home_score > away_score:
            # Ev sahibi kazandı
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'W'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'Pts'] += 3
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'L'] += 1
        elif home_score < away_score:
            # Deplasman kazandı
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'W'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'Pts'] += 3
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'L'] += 1
        else:
            # Beraberlik
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'D'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'Pts'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'D'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'Pts'] += 1

    # Puan durumunu puanlara göre sıralama
    puan_durumu = puan_durumu.sort_values(by=['Pts', 'GF', 'GA'], ascending=[False, False, True])
    puan_durumu["season_id"]=season_id
    
    return puan_durumu

# Sezon ve turnuva için puan durumu
puan_tablosu = puan_durumu(df, season_id=	61667)

puan_tablosu 


,team_id,W,D,L,GF,GA,Pts,season_id
4,2967,1,0,0,3,1,3,61667
2,2970,1,0,0,2,0,3,61667
1,2974,1,0,0,2,1,3,61667
3,46160,0,1,0,0,0,1,61667
7,2966,0,1,0,0,0,1,61667
5,2954,0,0,1,1,2,0,61667
0,46168,0,0,1,1,3,0,61667
6,2994,0,0,1,0,2,0,61667


In [2]:
#verileri oku
from basemodels.models import FikstureModelData
import pandas as pd

data = FikstureModelData.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df_exploded = df.explode('data')
expanded_data = pd.json_normalize(df_exploded['data'])
expanded_data['tarih'] = list(df_exploded['tarih'])
expanded_data

,customId,winnerCode,hasGlobalHighlights,hasEventPlayerStatistics,hasEventPlayerHeatMap,detailId,crowdsourcingDataDisplayEnabled,id,startTimestamp,slug,...,homeTeam.gender,awayTeam.gender,homeScore.aggregated,awayScore.aggregated,homeScore.penalties,awayScore.penalties,tournament.uniqueTournament.primaryColorHex,tournament.uniqueTournament.secondaryColorHex,coverage,tarih
0,dsr,2.0,True,True,True,1.0,False,12437039,1729429200,manchester-city-wolverhampton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
1,NsU,1.0,True,True,True,1.0,False,12437032,1729438200,liverpool-chelsea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
2,hso,1.0,True,True,True,1.0,False,12437036,1729537200,nottingham-forest-crystal-palace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
3,tgbsBgb,1.0,True,True,True,1.0,False,12437847,1729425600,mallorca-rayo-vallecano,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
4,LgbsVgb,1.0,True,True,True,1.0,False,12437835,1729433700,leganes-atletico-madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,fYbsmYb,NaN,False,NaN,NaN,NaN,False,12872190,1729622700,connahs-quay-nomads-aberystwyth-town,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22
550,lQrsjlLb,NaN,False,NaN,NaN,NaN,False,12464047,1729638000,ecuador-u17-new-zealand-u17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22
551,fQrsXlpd,NaN,False,NaN,NaN,NaN,False,12464084,1729638000,dominican-republic-u17-nigeria-u17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22
552,wlpskEu,NaN,False,NaN,NaN,NaN,False,12464043,1729627200,colombia-u17-spain-u17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22


In [5]:
expanded_data.to_excel("FullData.xlsx")

In [4]:
list(expanded_data.columns)

['customId',
 'winnerCode',
 'hasGlobalHighlights',
 'hasEventPlayerStatistics',
 'hasEventPlayerHeatMap',
 'detailId',
 'crowdsourcingDataDisplayEnabled',
 'id',
 'startTimestamp',
 'slug',
 'finalResultOnly',
 'feedLocked',
 'isEditor',
 'tournament.name',
 'tournament.slug',
 'tournament.category.name',
 'tournament.category.slug',
 'tournament.category.sport.name',
 'tournament.category.sport.slug',
 'tournament.category.sport.id',
 'tournament.category.id',
 'tournament.category.country.alpha2',
 'tournament.category.country.alpha3',
 'tournament.category.country.name',
 'tournament.category.country.slug',
 'tournament.category.flag',
 'tournament.category.alpha2',
 'tournament.uniqueTournament.name',
 'tournament.uniqueTournament.slug',
 'tournament.uniqueTournament.category.name',
 'tournament.uniqueTournament.category.slug',
 'tournament.uniqueTournament.category.sport.name',
 'tournament.uniqueTournament.category.sport.slug',
 'tournament.uniqueTournament.category.sport.id',
 

In [8]:
Seasonx = Season.objects.all().values()
Seasonx = list(Seasonx)
Seasonx = pd.DataFrame(Seasonx)
Seasonx

,season_id,season_name,season_year,tournament_id
0,57284,Allsvenskan 2024,2024,40
1,57318,USL League One 2024,2024,13362
2,57322,Eliteserien 2024,2024,20
3,57356,1st Division 2024,2024,22
4,57359,"Toppserien, Women 2024",2024,201
...,...,...,...,...
99,65687,Primera Division Women 24/25,24/25,1127
100,65961,Indian Super League 24/25,24/25,1900
101,66456,UEFA Youth League 24/25,24/25,2132
102,66662,Super League 2 24/25,24/25,186


In [9]:
Teamdata = Team.objects.all().values()
Teamdata = list(Teamdata)
Teamdata= pd.DataFrame(Teamdata)
Teamdata

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id,season_id
0,3,Wolverhampton,wolverhampton,Wolves,WOL,False,17,61627
1,7,Crystal Palace,crystal-palace,Crystal Palace,CRY,False,17,61627
2,14,Nottingham Forest,nottingham-forest,Forest,NFO,False,17,61627
3,17,Manchester City,manchester-city,Man City,MCI,False,17,61627
4,38,Chelsea,chelsea,Chelsea,CHE,False,17,61627
...,...,...,...,...,...,...,...,...
654,826802,Fiorentina U20,acf-fiorentina-u20-u20,Fiorentina U20,ACF,False,2292,64432
655,826810,Udinese U20,udinese-calcio-u20-u20,Udinese U20,UDI,False,2292,64432
656,826812,Genoa U20,genoa-cfc-u20-u20,Genoa U20,GEN,False,2292,64432
657,826817,Sampdoria U20,sampdoria-genoa-u20-u20,Sampdoria U20,SAM,False,2292,64432


In [15]:
takım = Teamdata[Teamdata["team_id"]==3052]
takım

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id,season_id
200,3052,Fenerbahçe,fenerbahce,Fenerbahçe,FEN,False,52,63814


In [17]:
from basemodels.models import Tournament, Season, Team
Tournamentx = Tournament.objects.all().values()
Tournamentx = list(Tournamentx)
Tournamentx= pd.DataFrame(Tournamentx)
Tournamentx

,tournament_id,tournament_name,tournament_slug,tournament_category_name,tournament_category_slug,tournament_uniqueTournament_category_id,tournament_uniqueTournament_hasEventPlayerStatistics,tournament_model_id,tournament_isGroup,tournament_uniqueTournament_hasPerformanceGraphFeature
0,8,LaLiga,laliga,Spain,spain,32,False,36,False,False
1,9,Challenger Pro League,challenger-pro-league,Belgium,belgium,33,False,692,False,False
2,17,Premier League,premier-league,England,england,1,False,1,False,False
3,18,Championship,championship,England,england,1,False,2,False,False
4,20,Eliteserien,eliteserien,Norway,norway,5,False,5,False,False
...,...,...,...,...,...,...,...,...,...,...
99,11621,"Liga MX, Apertura",liga-mx-apertura,Mexico,mexico,12,False,28,False,False
100,11653,Primera División,primera-division,Chile,chile,49,False,64475,False,False
101,13362,"USL, League One",usl-league-one,USA,usa,26,False,71366,False,False
102,14688,Super Cup,super-cup,Egypt,egypt,305,False,22348,False,False
